#### things to make modular
* query expansion
* query expansion result aggregation method
* using clean corpus
* retrieval method (dres/bm25)

#### imports

In [1]:
from coir.data_loader import get_tasks
from coir.evaluation import COIR
from coir.models import YourCustomDEModel
from datasets import load_dataset, Dataset, concatenate_datasets
import pandas as pd
from query_expansion import expand_queries_and_qrels
from functions import * 
from collections import defaultdict

In [2]:
cosqa_expl_path = '/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/Cosqa/CoSQA_explanations_query_code.csv'
cosqa_deepseek_updated_path='/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/CS696DS-Oracle-Retrieving-Code-Explanations/Explanation_Generation/Cosqa/postprocessing/output/COSQA_deepseek_explanations_clean.csv'
cosqa_granite_updated_path='/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/CS696DS-Oracle-Retrieving-Code-Explanations/Explanation_Generation/Cosqa/postprocessing/output/COSQA_granite_explanations_clean.csv'
csn_expl_path = '/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/explanations/CodeSearchNet_Python_deepseek_allSplits.csv'

#### Query Expansion

In [12]:
cosqa_deepseek_temp_0_qe_path = '/work/pi_wenlongzhao_umass_edu/27/janet/query_expansion/results/cosqa/cosqa_queries_expanded_deepseek_temp_0.csv'
cosqa_queries_expanded_granite_temp_0pt5_mmr_2_path='/work/pi_wenlongzhao_umass_edu/27/janet/query_expansion/results/cosqa/cosqa_queries_expanded_granite_temp_0.5_mmr_2.csv'
cosqa_qe_df = pd.read_csv(cosqa_deepseek_temp_0_qe_path)
cosqa_qe_df = cosqa_qe_df.rename(columns={'mmr_explanation': 'expanded_query'})
cosqa_qe_df.head()
dataset_name = "cosqa"
tasks = load_data(dataset_name)
corpus, queries, qrels = tasks[dataset_name]

# updating the corpus with no comments in code
df= pd.read_csv(cosqa_deepseek_updated_path)
corpus = update_corpus_with_cleaned_code(corpus, df) 

# updating queries for qe
new_queries, new_qrels = expand_queries_and_qrels(cosqa_qe_df, qrels)
tasks[dataset_name] = (corpus, new_queries, qrels)

in tasks 
fetched data from hf
in deduplicate_qrels_by_code
data loader init


0it [00:00, ?it/s]

#### No QE

In [13]:
# dataset_name = "cosqa"
# tasks = load_data(dataset_name)
# corpus, queries, qrels = tasks[dataset_name]

In [14]:
# df= pd.read_csv(cosqa_deepseek_updated_path)
# corpus = update_corpus_with_cleaned_code(corpus, df) # updating the corpus with no comments in code
# tasks[dataset_name] = (corpus, queries, qrels)

#### Running the pipeline

In [15]:
# if you want to add explanations (deepseek / granite)
#add_expl(tasks, "cosqa", cosqa_deepseek_updated_path, "explanation_deepseek_1_cleaned") # adding explanation instead of code

In [17]:
run("BAAI/bge-base-en", tasks, "baseline", "dres", "cosqa_qe/granite_temp_0.5_mmr_2/average")

YourCustomDEModel init
COIR init!
in evaluation.py: loading up dres

in evaluation.py: retrieving

in beir/retrieval/evaluation.py: loading up search

in exact_search.py
len of queries:  89817


Encoding batches:   0%|          | 0/351 [00:00<?, ?batch/s]

len of corpus:  20604


Encoding batches:   0%|          | 0/81 [00:00<?, ?batch/s]

in evaluation.py: evaluating

in beir/retrieval/evaluation.py: evaluating now

in combine scores by average
Retrieval evaluation results saved to results/cosqa_qe/granite_temp_0.5_mmr_2/average/baseline/dres/BAAI/bge-base-en
{'cosqa': {'NDCG': {'NDCG@1': 0.078, 'NDCG@3': 0.13912, 'NDCG@5': 0.17166, 'NDCG@10': 0.20211, 'NDCG@100': 0.27631, 'NDCG@1000': 0.3032}, 'MAP': {'MAP@1': 0.078, 'MAP@3': 0.12367, 'MAP@5': 0.14197, 'MAP@10': 0.15456, 'MAP@100': 0.16774, 'MAP@1000': 0.1688}, 'Recall': {'Recall@1': 0.078, 'Recall@3': 0.184, 'Recall@5': 0.262, 'Recall@10': 0.356, 'Recall@100': 0.726, 'Recall@1000': 0.936}, 'Precision': {'P@1': 0.078, 'P@3': 0.06133, 'P@5': 0.0524, 'P@10': 0.0356, 'P@100': 0.00726, 'P@1000': 0.00094}}}


#### duplicate checking

In [ ]:
duplicate_code_groups = df.groupby('code').filter(lambda x: len(x) > 1)
num_duplicate_code_groups = duplicate_code_groups['code'].nunique()
print(f"Number of code groups with >1 item: {num_duplicate_code_groups} out of {len(df)}")

In [ ]:
duplicate_code_df = df.groupby('code').filter(lambda x: len(x) > 1)

result = (
    duplicate_code_df
    .groupby('code')
    .agg({
        'corpus_id': lambda x: list(set(x)),
        'query_id': lambda x: list(set(x))
    })
    .reset_index()
    .rename(columns={'corpus_id': 'corpus_ids_with_same_code', 'query_id': 'query_ids_for_those_corpus'})
)

print(result.head())


                                                code  \
0  async def _send_plain_text(self, request: Requ...   
1  async def _thread_coro(self, *args):\n        ...   
2  async def async_input(prompt):\n    """\n    P...   
3  async def async_run(self) -> None:\n        ""...   
4  async def cursor(self) -> Cursor:\n        """...   

                           corpus_ids_with_same_code  \
0              [d3961, d1874, d20448, d6227, d10337]   
1  [d6144, d118, d5150, d12781, d7622, d1889, d82...   
2  [d2781, d16258, d16413, d4944, d9133, d9137, d...   
3           [d19465, d20020, d16870, d19314, d19313]   
4                   [d16659, d18387, d19386, d17979]   

                          query_ids_for_those_corpus  
0              [q20448, q1874, q3961, q6227, q10337]  
1  [q11702, q5150, q6147, q7622, q8299, q118, q18...  
2  [q4944, q16413, q9137, q16258, q20250, q9133, ...  
3           [q20020, q19314, q16870, q19465, q19313]  
4                   [q17979, q19386, q16659, q18387

In [ ]:
duplicate_code_groups = df.groupby('cleaned_code').filter(lambda x: len(x) > 1)

num_duplicate_code_groups = duplicate_code_groups['cleaned_code'].nunique()

print(f"Number of cleaned_code groups with >1 item: {num_duplicate_code_groups} out of {len(df)}")

Number of cleaned_code groups with >1 item: 3406 out of 20604


In [ ]:
duplicate_code_groups = df.groupby('explanation_deepseek_1_cleaned').filter(lambda x: len(x) > 1)

num_duplicate_code_groups = duplicate_code_groups['explanation_deepseek_1_cleaned'].nunique()

print(f"Number of explanation_deepseek_1_cleaned groups with >1 item: {num_duplicate_code_groups} out of {len(df)}")

Number of explanation_deepseek_1_cleaned groups with >1 item: 93 out of 20604
